<a href="https://colab.research.google.com/github/vipashaaV321/Collaborative-Food-Recipe-Recommendation-System/blob/Vipasha/SVD_MRR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = pd.read_csv("/content/drive/MyDrive/Data/RS2/short-recipes-20.csv")

In [4]:
# Create the utility matrix
utility_matrix = pd.pivot_table(data, values='rating', index='user_id', columns='recipe_id', fill_value=0)

In [5]:
# Normalize the ratings
mean_ratings = np.mean(utility_matrix, axis=1)
utility_matrix = utility_matrix.sub(mean_ratings, axis=0)

In [6]:
# Sparse Matrix
sparse_utility_matrix = csr_matrix(utility_matrix)

In [7]:
# Perform matrix factorization
u, s, vt = svds(sparse_utility_matrix, k=50)

In [8]:
# Generate recommendations
def get_recommendations(previous_selections):
    if len(previous_selections) == 0:
        print("Welcome to the food recommendation system! What kind of food do you like?")
    else:
        print("Based on your previous selections, here are some more recommendations:")
    recommended_items = recommend_items(previous_selections)
    for i, item in enumerate(recommended_items):
        print(f"{i+1}. {item}")
    return recommended_items

In [9]:
def recommend_items(previous_selections):
    # Get the indices of the previous selections
    indices = [data[data['name']==item]['recipe_id'].iloc[0] for item in previous_selections]

    # Predict the ratings for all the recipes
    predicted_ratings = np.dot(np.dot(u, np.diag(s)), vt)

    # Get the predicted ratings for the previous selections and all the recipes
    previous_ratings = predicted_ratings[:, indices].mean(axis=1)
    all_ratings = predicted_ratings[:, :].mean(axis=1)

    # Get the indices of the top 5 highest predicted ratings
    top_indices = np.argsort(all_ratings - previous_ratings)[-5:][::-1]

    # Get the recipe names from the indices
    top_items = [data[data['recipe_id']==index]['name'].iloc[0] for index in top_indices]

    return top_items

In [10]:
# Start the conversation
previous_selections = []
while True:
    # Get user input
    user_input = input("\nEnter a food item you like or type 'done' to exit: ")

    # Check if the user wants to exit
    if user_input.lower() == 'done':
        print("Thank you for using the food recommendation system!")
        break

    # Check if the user has already selected this item
    if user_input in previous_selections:
        print(f"You've already selected {user_input}. Please choose a different food item.")
        continue

    # Check if the item is in the dataset
    if user_input not in data['name'].values:
        print("Sorry, we don't have that item in our dataset. Please try a different food item.")
        continue

    # Add the item to the previous selections
    previous_selections.append(user_input)

    # Get the recommendations
    recommended_items = get_recommendations(previous_selections)


Enter a food item you like or type 'done' to exit: to die for crock pot roast	
Sorry, we don't have that item in our dataset. Please try a different food item.

Enter a food item you like or type 'done' to exit: creamy cajun chicken pasta
Sorry, we don't have that item in our dataset. Please try a different food item.

Enter a food item you like or type 'done' to exit: whatever floats your boat brownies
Sorry, we don't have that item in our dataset. Please try a different food item.

Enter a food item you like or type 'done' to exit: Apple Pie
Sorry, we don't have that item in our dataset. Please try a different food item.

Enter a food item you like or type 'done' to exit: all in the kitchen chili
Sorry, we don't have that item in our dataset. Please try a different food item.

Enter a food item you like or type 'done' to exit: alouette potatoes
Sorry, we don't have that item in our dataset. Please try a different food item.

Enter a food item you like or type 'done' to exit: done
Th